# Connecting to Twitter, Handling Tweets

## Imports

In [ ]:
import (
    "encoding/json"
    "net"
    "net/http"
    "net/url"
    "strconv"
    "strings"
    "sync"
    "time"
    "fmt"
    "os"
    
    "github.com/garyburd/go-oauth/oauth"
)

## Creating a tweet reader

We are going to define a couple types. First, we are going to utilize the `Tweet` struct that we have seen in prior notebooks, and then we are going to define a `TweetReader` struct type that will hold all the info we need when connecting to Twitter.

In [ ]:
// Tweet is a single tweet.
type Tweet struct {
    Text string
    Terms []string
}

// TweetReader includes the info we need to access Twitter.
type TweetReader struct {
    ConsumerKey, ConsumerSecret, AccessToken, AccessSecret string
}

Also, let's define a function `NewTweetReader` that will initialize a `TweetReader` value for us.

In [ ]:
// NewTweetReader creates a new TweetReader with the given credentials.
func NewTweetReader(consumerKey, consumerSecret, accessToken, accessSecret string) *TweetReader {
    return &TweetReader{
        ConsumerKey:    consumerKey,
        ConsumerSecret: consumerSecret,
        AccessToken:    accessToken,
        AccessSecret:   accessSecret,
    }
}

## Creating an HTTP client

Next, we are going to define a new HTTP client that we will use to retrieve our Tweets. Because we are going to do some streaming analysis and may be utilizing this client from mutliple goroutines, I have borrowed the HTTP client configuration from a similar MachineBox project that will keep us safe in these scenarios. It includes both a Mutex for accessing the client and some timeout functionality.

In [ ]:
// Create a new HTTP client.
var connLock sync.Mutex
var conn net.Conn
client := &http.Client{
    Transport: &http.Transport{
        Dial: func(netw, addr string) (net.Conn, error) {
            connLock.Lock()
            defer connLock.Unlock()
            if conn != nil {
                conn.Close()
                conn = nil
            }
            netc, err := net.DialTimeout(netw, addr, 5*time.Second)
            if err != nil {
                return nil, err
            }
            conn = netc
            return netc, nil
        },
    },
}

## Making a request for tweets

To retrieve Tweets from Twitter's streaming API, you will need to obtain a set of credentials from your Twitter account. To do this:

1. Go to apps.twitter.com
2. Create a new app
3. Under that app, under `Keys and Access Tokens` retrieve your connection key and secret.
4. Generate an access token and token secret on that same page.
5. Retrieve the access token and token secret.

We can now create our `TweetReader` value and an oauth client and credentials:

In [ ]:
// Create a new Tweet Reader.
consumerKey := ""
consumerSecret := ""
accessToken := ""
accessSecret := ""
r := NewTweetReader(consumerKey, consumerSecret, accessToken, accessSecret)

In [ ]:
// Create oauth Credentials.
creds := &oauth.Credentials{
    Token:  r.AccessToken,
    Secret: r.AccessSecret,
}

// Create an oauth Client.
authClient := &oauth.Client{
    Credentials: oauth.Credentials{
        Token:  r.ConsumerKey,
        Secret: r.ConsumerSecret,
    },
}

We are going to retrieve tweets by keywords contained in those tweets. Feel free to modify the keywords below according to your interests, but keep in mind that you may want to pick frequently used keywords to make sure something is returned:

In [ ]:
// Define the terms for our search.
terms := []string{"Trump", "Russia"}
form := url.Values{"track": terms}
formEnc := form.Encode()

Finally, we can create our request to the Twitter streaming API and use our client to execute that request:

In [ ]:
// Create a new HTTP request.
u, err := url.Parse("https://stream.twitter.com/1.1/statuses/filter.json")
if err != nil {
    fmt.Println("Could not parse url:", err)
}

req, err := http.NewRequest("POST", u.String(), strings.NewReader(formEnc))
if err != nil {
    fmt.Println("creating filter request failed:", err)
}

// Set some header info.
req.Header.Set("Authorization", authClient.AuthorizationHeader(creds, "POST", u, form))
req.Header.Set("Content-Type", "application/x-www-form-urlencoded")
req.Header.Set("Content-Length", strconv.Itoa(len(formEnc)))

In [ ]:
// Do the request.
resp, err := client.Do(req)
if err != nil {
    fmt.Println("Error getting response:", err)
}
if resp.StatusCode != http.StatusOK {
    fmt.Println("failed with status code:", resp.StatusCode)
}

## Reading retrieved tweets

Now to read successive tweets returned in the response, we are going to create a new decoder using the `encoding/json` package. This decoder will read from the provided reader. We will then loop 10 times reading in 10 tweets and output them to stdout.

In [ ]:
// Create a new decoder for the response body.
decoder := json.NewDecoder(resp.Body)

// Start reading in tweets and parsing them.
for i := 0; i < 10; i++ {
    var t Tweet
    if err := decoder.Decode(&t); err != nil {
        break
    }
    fmt.Printf("TWEET %d TEXT: %s\n", i+1, t.Text)
    fmt.Println("----------------------------------------\n")
}

// Close the response body.
resp.Body.Close()